In [1]:
# Step 1 - import pytorch
import torch
from torch import nn

In [2]:
# Step 2 - get device to run model on - this is the main difference from what Markus showed us
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [3]:
# Step 3 - define a simply neural network model
class OneLayerNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(8, 1)

    def forward(self, x):
        output = self.layer(x)
        return output

In [ ]:
# Step 4 - instantiate mode, and create a suitable input tensor
input = torch.Tensor([1]*8)
neural_network = OneLayerNN()

In [5]:
input

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

In [6]:
input.shape

torch.Size([8])

In [7]:
# You can see the weights here! If you recreate the tensor, the weights will change! This is because initialization is somewhat random!
list(neural_network.layer.named_parameters())

[('weight',
  Parameter containing:
  tensor([[-0.2851, -0.0767, -0.3202,  0.2581,  0.2895, -0.1815, -0.2110, -0.1919]],
         requires_grad=True)),
 ('bias',
  Parameter containing:
  tensor([-0.2113], requires_grad=True))]

In [8]:
# Step 5 - apply the network to your input tensor
# Note that this is running on your CPU!!!
neural_network(input)

tensor([-0.9300], grad_fn=<ViewBackward0>)

In [10]:
# Step 6 - try running it with apple mps acceleration
neural_network.to(device) # Send NN to mps
input = input.to(device) # Send input to mps
output = neural_network(input)

In [11]:
output.device # Note that the output is on the mps device too

device(type='mps', index=0)

In [ ]:
output # We can print the output ... the display string will tell us the values, as well as what device it's on

tensor([-0.9300], device='mps:0')

In [ ]:
# We can convert our tensors to a numpy array with the .numpy() method.
# However, to do so the tensor must first be on our CPU
# and 'detached' - which means that we've told pytorch it doesn't need
# to track the gradient for this tensor anymore... since it won't be used in backpropagation
# We do that with the .detach() method.
# To move it to the cpu, can use the .cpu() method, which is equivalent to .device('cpu')
output_numpy = output.cpu().detach().numpy()
output_numpy

array([-0.9299588], dtype=float32)